<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


In [10]:
!pip install openai --quiet

In [11]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [12]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [13]:
def call(system_prompt: str, message: str) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [14]:
def marketing_campaign_agent(product : str):
  """
     <PRODUCT>{product}</PRODUCT>

     Your task is to create an compelling marketing campaign for the product provided between the <product> tag.
  """
  return call ("your are a marketing campaign export.", marketing_campaign_agent.__doc__.format(product=product) )

In [18]:
def ready_validation_agent(marketing_text : str) -> bool:
  """

    <CONTENT>{text}</CONTENT>
    I have asked you to create a marketing text for a product  earlier. You can find the text between the <CONTENT> tags.

    Is this the final version of the marketing text and ready to publish or can we improve it?

     Your task is to answer with
     - TRUE if this is the final version of the marketing text and ready to publish.
     - FALSE if this is not the final version of the marketing text and we need to improve it.

     Only answer with TRUE or FALSE. add nothing else to your response.

    """
  result = call ("your are a marketing campaign export.", ready_validation_agent.__doc__.format(text=marketing_text) )
  print (result)
  return result.strip().lower() == "true"

In [16]:
def refinements_agent(product : str, marketing_text : str) -> str:
  """
  <PRODUCT>{product}</PRODUCT>

  <CONTENT>{text}</CONTENT>

  I have asked you to create a marketing text for a product earlier. You can find the text between the <CONTENT> tags.

  This marketing text is not final and not yet ready for publishing.

  Your task is to analyse the text and think about improvments that we can do the text on order to finalize it.

  ### OUTPUT
  Please output a  list of improvements you have found.

  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text) )

In [25]:
def refine_agent(product : str, marketing_text : str, refinements : str ) -> str:
  """
  <PRODUCT>{product}</PRODUCT>

  <CONTENT>{text}</CONTENT>

  I have asked you to create a marketing text for a product earlier. You can find the text between the <CONTENT> tags.

  Your task is to refine the marketing text provided betwen the <CONTENT> tag with the follwing refinements. Don't use any other sources.

 ### Refinements
 {refinements}


  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text, refinements=refinements) )

In [ ]:
product = "iPhone 15"

marketing_text = marketing_campaign_agent(product)
is_finished = ready_validation_agent(marketing_text)
refinements = refinements_agent(product, marketing_text)
next_text_iteration = refine_agent(product, marketing_text, refinements)


FALSE


In [27]:
print (marketing_text)
print ("###### critique")
print (is_finished)
print (refinements)
print ("###### next text iteration")
print (next_text_iteration)

**Marketing Campaign: "Elevate Your World"**

**Tagline:** Experience the future of smartphones with iPhone 15

**Objective:** Create a buzz around the launch of iPhone 15, emphasizing its cutting-edge features and innovative design while appealing to existing customers and new users alike.

**Target Audience:**

1. Young professionals (25-35 years old)
2. Tech-savvy individuals
3. Existing iPhone 14 users looking for an upgrade

**Key Messaging:**

* Unlock a world of endless possibilities with iPhone 15's advanced camera system, fast A17 Bionic chip, and long-lasting battery life.
* Experience the innovative MagSafe design, combining style and functionality.
* Discover new ways to express yourself with our stunning display and immersive augmented reality capabilities.

**Marketing Channels:**

1. **Social Media Campaigns:**
	* Facebook:
		+ Teaser ad series highlighting iPhone 15's key features
		+ Influencer partnerships with popular tech-focused content creators
	* Instagram:
		+ E